In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pwd

In [ ]:
!rm -rf /kaggle/working/*

In [ ]:
!git clone -q https://github.com/aryanda1/temp-voimo
!mv temp-voimo/* .
!rm -rf temp-voimo

In [ ]:
import environnew

In [ ]:
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone -q https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr -q
!pip install facexlib -q
!pip install gfpgan -q
!pip install -r requirements.txt -q
!python setup.py develop 
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
%cd ..

In [ ]:
!mkdir models

In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?export=download&id=12L9Z5fO748i8Ua9q0qZBhwwBJ0SyM0rg','models/model.ckpt')

In [ ]:
openai_api = input('Enter openai Api key - ')

In [ ]:
!rm -rf output/

In [ ]:
!mkdir output/

In [ ]:
!pip install scikit-image==0.19.2 -q

In [ ]:
from fastapi import FastAPI, Request, File, Form, BackgroundTasks
from fastapi.responses import FileResponse
from fastapi.middleware.cors import CORSMiddleware
import sys
import os
sys.path.extend([
    'src/taming-transformers',
    'src/clip',
    'stable-diffusion/',
    'k-diffusion',
    'pytorch3d-lite',
    'AdaBins',
    'MiDaS',
])
import load_model
import torch
from mainOld import musictoVid
import time
app = FastAPI()
origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

custom_checkpoint_path = 'models/model.ckpt'
models_path = "/content/models" 
os.makedirs(models_path, exist_ok=True)
model = load_model.model(models_path,custom_checkpoint_path=custom_checkpoint_path)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)

tasks = {}

@app.get('/')
async def home():
  return {'mes':'Welcome!'}


def long_running_task(device, model, audio_path, theme, artist, option, upscale, task_id):
    # Do the long-running task here
    video_path = musictoVid(device, model, audio_path, theme, artist, int(option), upscale,task_id,openai_api)
    tasks[task_id]['status'] = 'completed'
    tasks[task_id]['path'] = video_path

@app.post("/submit_form")
async def submit_form(background_tasks: BackgroundTasks,audio: bytes = File(...), artist: str = Form(''), theme: str = Form(''), option: str = Form(...), upscale: bool = Form(False)):
    # Save audio to current directory
    with open("audio.wav", "wb") as f:
        f.write(audio)
    
    # Print other form inputs
    # print(f"Text input 1: {theme}")
    # print(f"Text input 2: {artist}")
    # print(f"Radio button: {option}")
    # print(f"Checkbox: {upscale}")
    timestring = time.strftime('%Y%m%d%H%M%S')
    tasks[timestring] = {'status':'processing'}
    background_tasks.add_task(long_running_task, device, model, 'audio.wav', theme, artist, option, upscale, timestring)
    # return FileResponse(video_path)
    return {"id": timestring}

@app.post("/get_video/{task_id}")
async def get_video(task_id: str):
    # Retrieve the result of the task from the task dictionary using the task ID
    if task_id in tasks:
        task = tasks[task_id]
        status = task["status"]
        if status == "completed":
            video_path = task["path"]
            print(video_path)
            filename = video_path.split('/')[-1]
            headers = {"Content-Disposition": f'attachment; filename="{filename}"'} 
            return FileResponse(video_path, headers=headers)
        else:
            return {"status": status}
    else:
        return {"error": "Task not found"}

In [ ]:
!ngrok authtoken 2Va9RU4kMARr2iilVBVHcombtwJ_7xCddiKbbriweqiLsS5UK

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000, timeout_keep_alive=3600)

In [ ]:
jjj

In [ ]:
!pip list

In [ ]:
import os

In [ ]:
!ls output

In [ ]:
os.listdir('output')

In [ ]:
from spleeter import split

In [ ]:
!ls

In [ ]:
!ls

In [ ]:
split('audio.wav')